In [ ]:
from fatiando.gravmag import normal_gravity
from fatiando.vis import mpl
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
lon, lat, height, gravity = np.loadtxt('eigen-6c4-grav.gdf', skiprows=34,
                                       unpack=True)
topo = np.loadtxt('eigen-6c4-topo.gdf', skiprows=29, usecols=[-1], unpack=True)
shape = (53, 210)

area = (lon.min(), lon.max(), lat.min(), lat.max())

In [ ]:
# First, lets calculate the gravity disturbance (e.g., the free-air anomaly)
# We'll do this using the closed form of the normal gravity for the WGS84
# ellipsoid
gamma = normal_gravity.gamma_closed_form(lat, height)
disturbance = gravity - gamma

In [ ]:
# Now we can remove the effect of the Bouguer plate to obtain the Bouguer
# anomaly. We'll use the standard densities of 2.67 g.cm^-3 for crust and 1.04
# g.cm^-3 for water.
bouguer = disturbance - normal_gravity.bouguer_plate(topo)

In [ ]:
topo_positive = topo[topo >= 0]
topo_negative = topo[topo < 0]

In [ ]:
jacobian = np.zeros([len(bouguer),2])
jacobian[topo < 0, 0] = topo_negative
jacobian[topo >= 0, 1] = topo_positive

In [ ]:
y = bouguer

### Cálculo do parâmetro por mínimos quadrados

In [ ]:
a_est = np.dot(np.dot(np.linalg.inv(np.dot(jacobian.transpose(),jacobian)),jacobian.transpose()),y)
print('a estimado=',a_est)

### Cálculo do parâmetro por mínimos quadrados reponderado

In [ ]:
weight = np.zeros([len(y),len(y)])
np.fill_diagonal(weight,1./((y - np.dot(jacobian,a_est))**2)**0.5)

In [ ]:
aest=np.dot(np.dot(np.linalg.inv(np.dot(jacobian.transpose(),np.dot(weight,jacobian))),jacobian.transpose()),np.dot(weight,y))
print('a estimado reponderado',aest)